# Consolidados de informes de Participación Enseño Porque Quiero
Hola, este documento de *Google Colab* tiene como finalidad permitir realizar un archivo de consolidado de participación para los archivos extraídos desde Zoom de la estrategia Aprender es un Parche de Ude@. Para poder realizar esto, debemos tener dos carpetas especificas a la mano:

-	***Carpeta con todos los Informe de Asistencia de Zoom)***

La descarga de estos archivos se realiza desde el espacio de Zoom de **Apropiación Ude@**. Es recomendable que al momento de realizar los **informes de participación**, conserves estos archivos y dejes una copia en una carpeta aparte, de esta forma ahorraras tiempo al momento de utilizar esta función para los consolidados. Recuerda que los archivos deben tener un nombre reconocible que te permita discriminar la fecha y el tipo de archivo que es, como recomendación se tiene el formato de:

### ast_fecha_EPQ.csv

- Al inicio del archivo indica que es el informe de asistencia (ast), seguido de un guion bajo ( _ ).
- A continuación, ubica la fecha del encuentro, iniciando por el mes, día y año, todo separando por guiones bajos ( _ ).
- Por último, las iniciales de la estrategia, en este caso EPQ.
- Ejemplo: Para el archivo de Reporte de Asistentes de Reunión del 9 de Agosto de 2050 el archivo resultante seria el: ast_08_09_2050_EPQ.csv (Recuerda que el .csv es el tipo de archivo, por lo que no lo debes cambiar)

\
Una vez identificados los archivos, crea una carpeta en Google Colab dando clic derecho en la sección de archivos, ubicado en el panel izquierdo. Recuerda ponerle un nombre claro a la carpeta, puede ser simplemente *ast*. Arrastra los archivos correspondientes a la carpeta y una vez los tengas cargados puedes empezar con las siguientes celdas de código.


## 1. Instalación de Paquetes Necesarios

---


Para iniciar, debes correr la celda de código que se encuentra abajo. Esta celda realizara la instalación de dos paquetes necesarios para el procesamiento de los archivos.

- **Stringr**: Es un paquete que procesa texto, esto ayuda a suprimir los caracteres especiales y dejar solo en minúsculas algunas secciones necesarias de los archivos.
- **xlsx**: Es el paquete que permite generar un archivo final de Excel con todos los datos que requieras.

In [ ]:
install.packages("stringr")
install.packages("openxlsx")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rJava’, ‘xlsxjars’




## 2. Asignación de Archivos y Función de Generación

---

En la celda de abajo tenemos el codigo de la función, no es necesario que entinedas en principio como funciona, basta con que tengas claro como utilizarla.

Anterirmente cargamos la carpeta con los archivos, antes de correr la celda, debes modificar el nombre de la carpeta en la primera lineas de codigo

```
# direccion_docs
```

Por último, debes modificar el nombre del archivo final.

```
# nombre_archivo_final
```

Al igual que para los archivos de asistencia, es recomendable tener un formato especifico. Como recomendación tenemos el siguiente formato:

### consolidado_fecha_EPQ.xlsx

- Al inicio del archivo indica un nombre que permita saber que es el consolidado de los encuentros y un guión bajo ( _ ).
- Seguido, ubica la fecha de generación, iniciando por el mes, día y año, todo separando por guiones bajos ( _ ).
- Por último, las iniciales de la estrategia, en este caso EPQ.
- Ejemplo: Para el archivo de Consolidado de encuentros generado el 9 de Agosto de 2050 el archivo resultante seria el: consolidado_08_09_2050_AEUP.xlsx (Recuerda que el .xslx es el tipo de archivo, por lo que no lo debes cambiar)

\
Ahora ya puedes correr el codigo, el archivo final se encontrará en el panel de archivos con el nombre asignado. Puede que dicho archivo tarde un poco en aparecer, en caso de no verlo rápido, puedes dar clic en el boton de recargar los archivos.

In [ ]:
# Esteban Motta Ruiz - Auxiliar de Porgramación en Ude@
# Consolidados de informes de Zoom - 23 de marzo de 2022
# Consolidados de participación Enseño Porque Quiero
# esteban.motta@udea.edu.co


direccion_docs <- "/content/archivos" # Debes editar esta sección
nombre_archivo_final <- "/content/consolidad_09_14_2022_EPQ.xlsx" # Debes editar esta sección

consolidados_EPQ <- function(d1, nom){

  setwd(d1)
  nombres_arch <- list.files(pattern = "*.csv")

  reportes_zoom <- lapply(list.files(pattern = "*.csv"),
                          read.csv,
                          encoding = "UTF-8",
                          header = FALSE)

  lista_Ins_Vin <- list()
  lista_Ins_Und <- list()
  lista_Ast_Vin <- list()
  lista_Ast_Und <- list()

  for (p in 1:length(reportes_zoom)) {

       Rep_Ins_file_dt <- subset.data.frame(reportes_zoom[[p]][-1, -c(6, 8, 9, 10, 11, 14, 15)])

       library(stringr)
       for (i in 1:length(Rep_Ins_file_dt)) {
            Rep_Ins_file_dt[i] <- str_replace_all(Rep_Ins_file_dt[, i], " ", " ")
       }

       colnom <- c("Asist", "Reg.Nombre", "Nombre", "Apellido", "Correo", "Estado", "Vinculacion", "Unidad")
       names(Rep_Ins_file_dt) <- colnom

       Rep_Ins_file_dt$Asist[Rep_Ins_file_dt$Asist == "Sí"] <- 1
       Rep_Ins_file_dt$Asist[Rep_Ins_file_dt$Asist == "No"] <- 0

       Rep_Ins_file_fn <- Rep_Ins_file_dt[!duplicated(Rep_Ins_file_dt$Correo), ]

       Rep_Ins_file_fn$Asist <- as.numeric(Rep_Ins_file_fn$Asist)

       Ins_total <- subset(Rep_Ins_file_fn[-c(1, 2), ],
                           select = c(Asist, Vinculacion, Unidad))
       Ins_total$Asist <- 1

       lista_Ins_Vin[[p]] <- aggregate(Asist~Vinculacion,
                                       data = Ins_total,
                                       FUN = sum)
       lista_Ins_Und[[p]] <- aggregate(Asist~Unidad,
                                       data = Ins_total,
                                       FUN = sum)

       Rep_Ast_file_fn <- Rep_Ins_file_fn[Rep_Ins_file_fn$Asist > 0, ]

       lista_Ast_Vin[[p]] <- aggregate(Asist~Vinculacion,
                                       data = Rep_Ast_file_fn[-c(1, 2), ],
                                       FUN = sum)
       lista_Ast_Und[[p]] <- aggregate(Asist~Unidad,
                                       data = Rep_Ast_file_fn[-c(1, 2), ],
                                       FUN = sum)

  }

  lista_Und <- list(lista_Ins_Und,
                    lista_Ast_Und)

  lista_Vin <- list(lista_Ins_Vin,
                    lista_Ast_Vin)

  lista_Und_cons <- list()
  lista_Vin_cons <- list()

  for(g in 1:length(lista_Und)){

      lista_Und_cons[[g]]<- Reduce(function(x, y) merge(x,
                                                        y,
                                                        by = "Unidad",
                                                        all = TRUE),
                                   lista_Und[[g]])

  }

  nombres_col_con_Und <- c("Unidad", nombres_arch)

  names(lista_Und_cons[[1]]) <- nombres_col_con_Und
  names(lista_Und_cons[[2]]) <- nombres_col_con_Und

  lista_Und_cons<- rapply(lista_Und_cons,
                          f = function(x) ifelse(test = is.na(x),
                                                 yes = 0,
                                                 no = x),
                          how = "replace" )


  for(h in 1:length(lista_Vin)){

      lista_Vin_cons[[h]]<- Reduce(function(x, y) merge(x,
                                                        y,
                                                        by = "Vinculacion",
                                                        all = TRUE),
                                   lista_Vin[[h]])

  }

  nombres_col_con_Vin <- c("Vinculacion", nombres_arch)

  names(lista_Vin_cons[[1]]) <- nombres_col_con_Vin
  names(lista_Vin_cons[[2]]) <- nombres_col_con_Vin

  lista_Vin_cons<- rapply(lista_Vin_cons,
                          f = function(x) ifelse(test = is.na(x),
                                                 yes = 0,
                                                 no = x),
                          how = "replace" )

  library(openxlsx)

  archivo_final <- list("Consolidado_Inscritos_Unidad" = lista_Und_cons[[1]],
                        "Consolidado_Asistentes_Unidad" = lista_Und_cons[[2]],
                        "Consolidado_Inscritos_Vinculacion" = lista_Vin_cons[[1]],
                        "Consolidado_Asistentes_Vinculacion" = lista_Vin_cons[[2]])

  write.xlsx(archivo_final,
             nom,
             colNames = TRUE,
             rowNames = FALSE)
}

consolidados_EPQ(d1 = direccion_docs,
                 nom = nombre_archivo_final)




